In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import math
import sys

In [2]:
puzzle='4.129..752..3..8...7..8...6...1.3.621.5...4.373.6.8...6...2..3...7..1..489..651.7'

In [3]:
rows = 'ABCDEFGHI'
cols = '123456789'

def cross(a,b):
    return [s+t for s in a for t in b]

In [4]:
boxes= cross(rows, cols)

In [5]:
row_units = [cross(r,cols) for r in rows]
column_units = [cross(rows,c) for c in cols]
square_units = [cross(rs,cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

unitlist = row_units + column_units + square_units
units = dict((s,[u for u in unitlist if s in u]) for s in boxes)
peers = dict((s,set(sum(units[s],[]))-set([s])) for s in boxes)

In [6]:
def display(values):
    width=1+max(len(values[s])for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|'if c in '36' else '')for c in cols))
        if r in 'CF':print(line)
    return

In [7]:
def elimination(values):
    solved_values = [box for box in values.keys() if len(values[box])==1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values

In [8]:
def only_choice(values):
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

In [9]:
def grid_values(grid):
    assert len(grid) == 81, "Input grid must be a string length of 81 (9x9)"
    boxes = cross(rows,cols)
    return dict(zip(boxes,grid))

def grid_values_improved(grid):
    values = []
    all_digits = '123456789'
    for c in grid:
        if c == '.':
            values.append(all_digits)
        elif c in all_digits:
                values.append(c)
    assert len(values) == 81
    boxes = cross(rows,cols)
    return dict(zip(boxes,values))

In [10]:
def reduce_puzzle(values):
    stalled =False
    while not stalled:
        solved_values_before = len([box for box in values.keys() if len(values[box])==1])
        elimination(values)
        only_choice(values)
        solved_values_after = len([box for box in values.keys() if len(values[box])==1])
        stalled = solved_values_after == solved_values_before
        if len([box for box in values.keys() if len(values[box])==1])==0:
            return False
    return values

In [11]:
def search(values):
    values_reduced = reduce_puzzle(values)
    if not values_reduced:
        return False
    else:
        values=values_reduced
    if len([box for box in values.keys() if len(values[box])==1])==81:
        return values
    
    possibility_count_list = [(len(values[box]),box) for box in values.keys() if len(values[box])>1]
    
    possibility_count_list.sort()
    for(_,t_box_min) in possibility_count_list:
        for i_digit in values[t_box_min]:
            new_values = values.copy()
            new_values[t_box_min]=i_digit
            new_values = search(new_values)
            if new_values:
                return new_values
            
    return values

In [12]:
p1=grid_values_improved(puzzle)
result = search(p1)
display(result)

4 8 1 |2 9 6 |3 7 5 
2 5 6 |3 1 7 |8 4 9 
3 7 9 |5 8 4 |2 1 6 
------+------+------
9 4 8 |1 5 3 |7 6 2 
1 6 5 |9 7 2 |4 8 3 
7 3 2 |6 4 8 |9 5 1 
------+------+------
6 1 4 |7 2 9 |5 3 8 
5 2 7 |8 3 1 |6 9 4 
8 9 3 |4 6 5 |1 2 7 


In [13]:
print(result)

{'A1': '4', 'A2': '8', 'A3': '1', 'A4': '2', 'A5': '9', 'A6': '6', 'A7': '3', 'A8': '7', 'A9': '5', 'B1': '2', 'B2': '5', 'B3': '6', 'B4': '3', 'B5': '1', 'B6': '7', 'B7': '8', 'B8': '4', 'B9': '9', 'C1': '3', 'C2': '7', 'C3': '9', 'C4': '5', 'C5': '8', 'C6': '4', 'C7': '2', 'C8': '1', 'C9': '6', 'D1': '9', 'D2': '4', 'D3': '8', 'D4': '1', 'D5': '5', 'D6': '3', 'D7': '7', 'D8': '6', 'D9': '2', 'E1': '1', 'E2': '6', 'E3': '5', 'E4': '9', 'E5': '7', 'E6': '2', 'E7': '4', 'E8': '8', 'E9': '3', 'F1': '7', 'F2': '3', 'F3': '2', 'F4': '6', 'F5': '4', 'F6': '8', 'F7': '9', 'F8': '5', 'F9': '1', 'G1': '6', 'G2': '1', 'G3': '4', 'G4': '7', 'G5': '2', 'G6': '9', 'G7': '5', 'G8': '3', 'G9': '8', 'H1': '5', 'H2': '2', 'H3': '7', 'H4': '8', 'H5': '3', 'H6': '1', 'H7': '6', 'H8': '9', 'H9': '4', 'I1': '8', 'I2': '9', 'I3': '3', 'I4': '4', 'I5': '6', 'I6': '5', 'I7': '1', 'I8': '2', 'I9': '7'}
